<a href="https://colab.research.google.com/github/eedduu/TFG/blob/main/code/old_playgrounds/TFGMLPMHibrid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ucimlrepo
!pip install fastai
!pip install nbdev

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
from ucimlrepo import fetch_ucirepo
import numpy as np
import pandas as pd
from fastai.tabular.all import *
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from nbdev.showdoc import *
from fastai.vision.all import *
import torch.nn.functional as F
import torchvision.transforms.functional as FF
import numpy
import torchvision
from torchvision import *
import fastai
import fastcore

In [ ]:
breast_cancer_wisconsin_diagnostic = fetch_ucirepo(id=17)

# data (as pandas dataframes)
X = breast_cancer_wisconsin_diagnostic.data.features
y = breast_cancer_wisconsin_diagnostic.data.targets

# metadata
#print(breast_cancer_wisconsin_diagnostic.metadata)

# variable information
#print(breast_cancer_wisconsin_diagnostic.variables)


df=X.join(y)

#X_train, y_train, X_test, y_test = train_test_split(X, y, test_size=0.3)


# Define the categorical and continuous variables
cat_names = []
#cont_names = [f'feature_{i}' for i in range(1, 31)]
cont_names = []
cont_names += ['texture1','perimeter1','area1','smoothness1','compactness1','concavity1','concave_points1','symmetry1','fractal_dimension1','texture2','perimeter2','area2','smoothness2','compactness2','concavity2','concave_points2','symmetry2','fractal_dimension2','texture3','perimeter3','area3','smoothness3','compactness3','concavity3','concave_points3','symmetry3','fractal_dimension3']
dep_var = 'Diagnosis'

splits=RandomSplitter(valid_pct = 0.3)(X)

# Create TabularPandas
to = TabularPandas(
    df,
    procs=[Categorify, FillMissing, Normalize],
    cat_names=cat_names,
    cont_names=cont_names,
    y_names=dep_var,
    splits=splits
)

#INFORED: He hecho varias pruebas pero hay que tener en cuenta que el dataset es pequeño con lo que un tamaño grande es contraproducente
dls = to.dataloaders(128)

# Create and train the learner
#learn = tabular_learner(dls, layers = [32], opt_func=SGD, metrics=accuracy, cbs=[SaveModelCallback(), EarlyStoppingCallback(min_delta=0, patience=4)])

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from fastai.tabular.all import TabularDataLoaders, TabularPandas, RandomSplitter, accuracy

# Define a simple neural network architecture
#

# Define the fitness function
def evaluate_fitness(weights, train_dl, valid_dl, dls):
    learn = tabular_learner(dls, layers=[64,32, 32])
    learn.model.load_state_dict(weights)
    learn.train()
    preds, targ, losses = learn.get_preds(dl=dls.train, with_loss=True)

    return losses.mean().item()

# Genetic Algorithm operators
def crossover(parent1, parent2):
    child = {}
    for key in parent1.keys():
        mask = torch.randint(0, 2, parent1[key].shape)
        child[key] = torch.where(mask ==1, parent1[key], parent2[key])
    return child

def mutate(weights, mutation_rate=0.01):
    mutated_weights = {}
    for key in weights.keys():
        mask = torch.rand(weights[key].shape) < mutation_rate
        mutation = torch.normal(0, 0.1, size=weights[key].shape)
        mutated_weights[key] = torch.where(mask, weights[key] + mutation, weights[key])
    return mutated_weights

def applyMemetic(weights, train_dl, valid_dl, dls):
    learn = tabular_learner(dls, layers=[64,32, 32])
    learn.model.load_state_dict(weights)
    learn.fit(1)
    learn.train()
    preds, targ, losses = learn.get_preds(dl=dls.train, with_loss=True)


    return learn.model.state_dict(), losses.mean().item()

# Load data
# Assuming you have your tabular data ready and named 'df'
# Define your input_size, hidden_size, and output_size based on your data
input_size = len(df.columns) -1   # Assuming the last column is the target
hidden_size = 32
output_size = 2  # Number of classes

# Prepare data loaders
splits = RandomSplitter()(range_of(df))
cat_names = []  # List your categorical columns here if any
cont_names = ['texture1','perimeter1','area1','smoothness1','compactness1','concavity1','concave_points1','symmetry1','fractal_dimension1','texture2','perimeter2','area2','smoothness2','compactness2','concavity2','concave_points2','symmetry2','fractal_dimension2','texture3','perimeter3','area3','smoothness3','compactness3','concavity3','concave_points3','symmetry3','fractal_dimension3'] # List your continuous columns here
dep_var = 'Diagnosis'

splits = RandomSplitter()(range_of(df))
to = TabularPandas(df, procs=[Categorify, FillMissing, Normalize],
                   cat_names=cat_names, cont_names=cont_names,
                   y_names=dep_var, splits=splits)
dls = to.dataloaders(bs=64)
train_dl, valid_dl = dls.train, dls.valid

# Initialize population. Inicializo las redes y cojo los pesos.
population_size = 25
population = [tabular_learner(dls, layers=[64, 32, 32]).model.state_dict() for _ in range(population_size)]

# EA loop
fitness_scores = [evaluate_fitness(weights, train_dl, valid_dl, dls) for weights in population]
num_generations = 200
for generation in range(num_generations):
    # Evaluate fitness


    # Selection (tournament selection)
    selected_indices_1 = np.random.choice(range(population_size), size=3, replace=False)
    selected_indices_2 = np.random.choice(range(population_size), size=3, replace=False)

    selected_fitness_scores_1 = [fitness_scores[i] for i in selected_indices_1]
    selected_fitness_scores_2 = [fitness_scores[i] for i in selected_indices_2]

    best1_index = selected_indices_1[np.argmin(selected_fitness_scores_1)] # Find the index of the best individual within the selected group
    best2_index = selected_indices_2[np.argmin(selected_fitness_scores_2)]

    parent1 = population[best1_index]
    parent2 = population[best2_index]


    # Crossover
    child = crossover(parent1, parent2)

    # Selection (tournament selection)
    selected_indices_1 = np.random.choice(range(population_size), size=3, replace=False)
    selected_indices_2 = np.random.choice(range(population_size), size=3, replace=False)

    selected_fitness_scores_1 = [fitness_scores[i] for i in selected_indices_1]
    selected_fitness_scores_2 = [fitness_scores[i] for i in selected_indices_2]

    best1_index = selected_indices_1[np.argmin(selected_fitness_scores_1)] # Find the index of the best individual within the selected group
    best2_index = selected_indices_2[np.argmin(selected_fitness_scores_2)]

    parent1 = population[best1_index]
    parent2 = population[best2_index]


    # Crossover
    child2 = crossover(parent1, parent2)

    # Mutation
    mutated_child = mutate(child)

    # Mutation
    mutated_child2 = mutate(child2)

    fit1= evaluate_fitness(mutated_child, train_dl, valid_dl, dls)
    fit2= evaluate_fitness(mutated_child2, train_dl, valid_dl, dls)

    best_child = mutated_child if fit1 < fit2 else mutated_child2
    best_fitness = fit1 if fit1 < fit2 else fit2

    # Replace the worst individual in the population with the mutated child
    if (best_fitness<np.max(fitness_scores)):
      worst_index = np.argmax(fitness_scores)
      population[worst_index] = best_child
      fitness_scores[worst_index] = best_fitness

    if(generation%50==0):
      #selected = np.random.choice(range(population_size), size=5, replace=False) -> 0.096
      selected = np.argsort(fitness_scores)[:5]
      for i in selected:
        population[i], fitness_scores[i] = applyMemetic(population[i], train_dl, valid_dl, dls)


# Get the best individual (set of weights)
best_individual_index = np.argmin(fitness_scores)
best_individual = population[best_individual_index]
#print("Best individual (weights):", best_individual)
print("Best individual (fitness):", fitness_scores[best_individual_index])


epoch,train_loss,valid_loss,time
0,0.470394,0.603606,00:00


epoch,train_loss,valid_loss,time
0,0.526152,0.584123,00:00


epoch,train_loss,valid_loss,time
0,0.492275,0.599155,00:00


epoch,train_loss,valid_loss,time
0,0.531002,0.608216,00:00


epoch,train_loss,valid_loss,time
0,0.602686,0.660481,00:00


...

epoch,train_loss,valid_loss,time
0,0.138393,0.123067,00:00


epoch,train_loss,valid_loss,time
0,0.105349,0.129073,00:00


epoch,train_loss,valid_loss,time
0,0.139236,0.106516,00:00


epoch,train_loss,valid_loss,time
0,0.128985,0.120379,00:00


Best individual (fitness): 0.08751095086336136


Nota: los errores de train los mostrados en los gráficos del entrenamientos de fastai y los calculados diferirán un poco, siendo más pequeños los calculados manualmente, ya que los calculados durante el entrenamiento hacen una media ponderada con valores donde el modelo era peor, obteniendo peores resultados. Los errores se calculan luego manualmente para tener un criterio objetivo y unificado, ya que ésta es la manera en la que se calcula el error cuando no se realiza entrenamiento GradDesc y simplemente se modifican los pesos.

Cada 50 generaciones se aplica GradDesc ya que si se usara más entonces no se vería el efecto del algoritmo generacional.

Tras varias pruebas, poblacion 25 y 200 iteraciones. Más iteraciones sería demasiado para las situacioines más grandes, y no hace falta tanta diversidad, se obtienen buenos resultados con 25 individuos.

Selecciono 5 individuos para aplicarle el Memetico pq es un 20% de la poblacion. Numero usual a nivel teorico. Tras probar hacerlo aleatorio y a los 5 mejores, se obtienen mejores resultados cogiendo a los 5 mejores, así que ese uso.

In [ ]:
fitness_scores

[0.08964917808771133,
 0.0996585339307785,
 0.09159177541732788,
 0.09352266043424606,
 0.09385485202074051,
 0.1000540554523468,
 0.10318837314844131,
 0.0977928563952446,
 0.09524211287498474,
 0.08849243819713593,
 0.10019384324550629,
 0.09584029763936996,
 0.09387154877185822,
 0.0932600125670433,
 0.09467329829931259,
 0.1021539568901062,
 0.10425326973199844,
 0.08751095086336136,
 0.10277232527732849,
 0.10059342533349991,
 0.09225060790777206,
 0.10340452939271927,
 0.0962584987282753,
 0.09925380349159241,
 0.09530439972877502]